<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Generate/ProtXLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Generate protein sequences using ProtXLNet pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 778kB 4.9MB/s 
     |████████████████████████████████| 3.0MB 24.8MB/s 
     |████████████████████████████████| 1.1MB 51.6MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 


In [2]:
import torch
from transformers import XLNetLMHeadModel, XLNetTokenizer,pipeline
import re
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtXLNet Model<b>

In [3]:
tokenizer = XLNetTokenizer.from_pretrained("Rostlab/prot_xlnet", do_lower_case=False)

In [4]:
model = XLNetLMHeadModel.from_pretrained("Rostlab/prot_xlnet")

<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
model = model.to(device)
model = model.eval()

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [7]:
sequences_Example = "A E T C Z A O"

In [8]:
sequences_Example = re.sub(r"[UZOB]", "X", sequences_Example) 

<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [9]:
ids = tokenizer.encode(sequences_Example, add_special_tokens=False)

In [10]:
input_ids = torch.tensor(ids).unsqueeze(0).to(device)

<b>6. Generate Protein Sequence<b>

In [11]:
max_length = 100
temperature = 1.0
k = 0
p = 0.9
repetition_penalty = 1.0
num_return_sequences = 3

In [12]:
output_ids = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=k,
        top_p=p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        num_return_sequences=num_return_sequences,
    )

In [13]:
output_sequences = [" ".join(" ".join(tokenizer.decode(output_id)).split()) for output_id in output_ids]

In [14]:
print('Generated Sequences\n')
for output_sequence in output_sequences:
  print(output_sequence)

Generated Sequences

A E T C X A X A A E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C I A K A G E T C
A E T C X A X V L E L F L L R D L G A E E M P E E V K K P D L G M L D E T I A D P K V V G T V V A L A V G D G S A Q E D P V R Q E R L L Q S S I G G E V A A A L A L A K M V M S L G E R S F R S E G G T
A E T C X A X V A S A D E P R L A K T A L G V L V G Q K S V V L S F E G T D Q A R L P P D V A T K V L E S M A Q R F G A E I P Q P V V T P A A L A L G L P S F T P T I I L A L P K L M S M P T G L L E L
